# Détection de fraude par carte de crédit

La fraude par carte de crédit se produit lorsque quelqu'un utilise les informations de carte de crédit ou de compte d'une autre personne pour effectuer des achats non autorisés ou accéder à des fonds par des avances de fonds. La fraude par carte de crédit ne se limite pas aux transactions en ligne; elle se produit également dans les magasins physiques. En tant que propriétaire d'entreprise, vous pouvez éviter de graves tracas – et une mauvaise publicité – en reconnaissant une utilisation potentiellement frauduleuse des cartes de crédit dans votre environnement de paiement.


# Problème
Le problème de détection de fraude par carte de crédit consiste à modéliser les transactions passées de cartes de crédit en connaissant celles qui se sont avérées être frauduleuses. Ce modèle est ensuite utilisé pour identifier si une nouvelle transaction est frauduleuse ou non. Notre objectif ici est de détecter 100 % des transactions frauduleuses tout en minimisant les classifications incorrectes de fraude.

# Observation
 - Très peu de transactions sont réellement frauduleuses (moins de 1%). L'ensemble de données est fortement déséquilibré, comprenant 492 fraudes sur un total de 284 807 observations. Cela a entraîné seulement 0,172 % de cas de fraude. Cet ensemble déséquilibré est justifié par le faible nombre de transactions frauduleuses.
 - L'ensemble de données se compose de valeurs numériques provenant des 28 caractéristiques transformées par l'« Analyse en Composantes Principales (PCA) », à savoir V1 à V28. De plus, aucune métadonnée sur les caractéristiques d'origine n'est fournie, ce qui empêche toute analyse préalable ou étude des caractéristiques.
 - Les caractéristiques ‘Time’ et ‘Amount’ ne sont pas des données transformées.
 - Il n'y a pas de valeurs manquantes dans l'ensemble de données.

**Importation des bibliothèques**

In [ ]:
# Importation des bibliothèques nécessaires

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ignorer les avertissements de future
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.warn('ignore', FutureWarning)

# Configurer l'affichage des graphiques dans le notebook
%matplotlib inline

# Appliquer un style de graphique propre

sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv('carte_de_credit..csv') # Chargement du fichier CSV contenant les données
df.head()

# Analyse exploratoire des données


In [ ]:
df.info() # Obtenir un résumé des informations du DataFrame

In [ ]:
# Réduire le nombre de décimales affichées dans le DataFrame
pd.set_option("display.float", "{:.2f}".format)
df.describe()

Check the missing value in dataset

In [ ]:
df.isnull().sum().sum() # Compter le nombre total de valeurs manquantes dans le DataFrame


In [ ]:
df.columns

# Les seules variables non transformées avec lesquelles travailler sont :
- Temps
- Montant
- Classe (1 : fraude, 0 : pas de fraude)

In [ ]:
labels = ["Sûr", "Frauduleux"]

count_classes = pd.value_counts(df['Classe'], sort=True)
count_classes.plot(kind='bar', rot=0)
plt.title("Répartition des Classes de Transactions")
plt.xticks(range(2), labels)
plt.xlabel("Classe")
plt.ylabel("Fréquence")

Ici, nous pouvons voir que, dans ce jeu de données, très peu de transactions sont réellement frauduleuses.

In [ ]:
df['Classe'].value_counts()

Notez à quel point notre jeu de données d'origine est déséquilibré ! La plupart des transactions ne sont pas frauduleuses. Si nous utilisons ce DataFrame comme base pour nos modèles prédictifs et nos analyses, nous risquons d'obtenir de nombreuses erreurs, et nos algorithmes risquent de surajuster car ils "supposeront" que la plupart des transactions ne sont pas frauduleuses. Mais nous ne voulons pas que notre modèle fasse des suppositions, nous voulons que notre modèle détecte les motifs qui signalent une fraude !

## Analyse statistique

- For dealing with outilers, IQR(Inter Quanrtile Range) in which we will eliminate the outliers those are less than 10th percentile greater than 90th percentile.

In [ ]:
Q1 = df.quantile(0.25)
Q2 = df.quantile(0.75)
IQR = Q2-Q1
print("IQR de l'ensemble du jeu de données :")
print(IQR)

In [ ]:
print("Asymétrie des données :")
df_skew = df.skew()
print(df_skew)

## Box Plot of the data

In [ ]:
print("Détection des valeurs aberrantes :\n  ")
plt.figure(figsize=(12,8))
plt.boxplot(df, vert=False)
plt.title("Détection des valeurs aberrantes")
plt.show()

- Temps et Montant : Les axes y affichent des valeurs aberrantes, vérifions cela.
Remarque : Les colonnes autres que Temps, Montant et Classe contiennent des nombres dans une certaine plage, nous pouvons les traiter lors de l'étape de prétraitement.

In [ ]:
def box_out(df):
    print("Détection des valeurs aberrantes :\n ")
    plt.figure(figsize=(12,8))
    plt.boxplot(df, vert=False)
    plt.title("Détection des valeurs aberrantes")
    plt.show()
    
print("Colonne Temps")
box_out(df['Temps'])

- Il n'y a aucun problème avec la colonne Temps.

Analyser la fonctionnalité **Montant** pour une meilleure compréhension du Montant.

In [ ]:
df['Montant'].describe()

La description ci-dessus nous montre que la plage du montant maximal et minimal des transactions est comprise entre 0 et 25 691.

### Diagramme en boîte sur le montant pour identifier les valeurs aberrantes de cette colonne

In [ ]:
print("Valeurs aberrantes du Montant :")
box_out(df['Montant'])      

- Il y a une bonne quantité de valeurs aberrantes dans la colonne "Montant". La colonne "Montant" est très importante dans ces données.
- L'image ci-dessus montre que, après 900, il y a des valeurs aberrantes qui sont réparties jusqu'à plus de 25 000+.
# Suppression des valeurs aberrantes avec
## Plafonnement et plancher basé sur les quantiles

In [ ]:
# Percentiles 
print("10e percentile du Montant : ")
print(df["Montant"].quantile(0.10))
print("90e percentile du Montant : ")
print(df["Montant"]. quantile(0.90))


- D'après les percentiles (10 et 90), nous pouvons supprimer les points de données qui sont en dehors de cette plage, mais il se peut qu'il n'y ait pas de valeurs aberrantes sous le 10e percentile.
- Ainsi, supprimons les valeurs aberrantes qui sont supérieures au 90e percentile et traçons un boxplot pour voir s'il y a des valeurs aberrantes inférieures au 10e percentile.

In [ ]:
df['Montant'] = np.where(df['Montant']>203.0, 203.0, df['Montant'])

print("Après avoir supprimé les outliers >90e percentile :")
box_out(df['Montant'])

- Nous avons éliminé la plupart des valeurs aberrantes supérieures au 90e percentile des données.
- Gardons la queue restante, car elle contient une bonne quantité de points.

In [ ]:

sns.lmplot(data=df, x='Temps', y='Montant', hue='Classe', fit_reg=False)


fig = plt.gcf()
fig.set_size_inches(12, 8)
plt.show()


In [ ]:
Frauduleux = df[df['Classe'] == 1]
Sûr = df[df['Classe'] == 0]

print(f"Forme des transactions frauduleuses : {Frauduleux.shape}")
print(f"Forme des transactions non frauduleuses : {Sûr.shape}")


Comment diffèrent les montants d'argent utilisés dans les différentes classes de transactions ?

In [ ]:
pd.concat([Frauduleux.Montant.describe(), Sûr.Montant.describe()], axis=1)

Les transactions frauduleuses se produisent-elles plus fréquemment à certains moments ?

In [ ]:
pd.concat([Frauduleux.Temps.describe(), Sûr.Temps.describe()], axis=1)

In [ ]:
plt.figure(figsize=(10,8))

plt.subplot(2, 2, 1)
plt.title('Distribution du Temps (Secondes)')

sns.distplot(df['Temps'], color='blue');

#plot the amount feature
plt.subplot(2, 2, 2)
plt.title('Distribution du Montant')
sns.distplot(df['Montant'],color='green');

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
df[df.Class == 1].Time.hist(bins=35, color='blue', alpha=0.6, label="Transaction Frauduleuse")
plt.legend()

plt.subplot(2, 2, 2)
df[df.Classe == 0].Time.hist(bins=35, color='green', alpha=0.6, label="Transaction Non Frauduleuse")
plt.legend()

En observant les distributions, nous pouvons avoir une idée de la façon dont ces caractéristiques sont biaisées, et nous pouvons également examiner les distributions des autres caractéristiques. Il existe des techniques qui peuvent aider à réduire l'asymétrie des distributions, et ces techniques seront mises en œuvre dans ce cahier dans le futur.

Il ne semble pas que l'heure de la transaction soit un facteur important ici, d'après les observations ci-dessus. Maintenant, prenons un échantillon du jeu de données pour notre modélisation et nos prédictions.

In [ ]:
df.hist(figsize=(20,20))

In [ ]:
# df.corr()

In [ ]:
#Trouvons les corrélations élevées

plt.figure(figsize=(10,8))
sns.heatmap(data=df.corr(), vmin=0,vmax=0.5, annot=False)
plt.show()

## Les corrélations les plus élevées proviennent de :

- Temps et V3 (-0.42)
- Montant et V2 (-0.53)
- Montant et V4 (0.4)


- Bien que ces corrélations soient élevées, je ne m'attends pas à ce qu'elles présentent un risque de multicolinéarité.

- La matrice de corrélation montre également qu'aucun des composants PCA de V1 à V28 n'a de corrélation entre eux. Cependant, si nous observons la classe, elle présente certaines corrélations positives et négatives avec les composants V, mais n'a aucune corrélation avec le Temps et le Montant.

# Traitement des données

Le temps et le montant doivent être mis à l'échelle comme les autres colonnes.

Division du jeu de données en ensembles d'entraînement et de test

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
print(X.shape, X_train.shape, X_test.shape)

Standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


# Entraînement du Modèle de Machine Learning
Il s'agit d'une tâche supervisée de Machine Learning avec un problème de classification.

# XGBoost

(Si le modèle ne s'entraîne pas bien avec les algorithmes ci-dessus, nous pouvons entraîner avec ANN)

In [ ]:
from xgboost import XGBClassifier
print("XGBoost fonctionne!")
from sklearn.ensemble import RandomForestClassifier 

## Classificateur XGBoost

In [ ]:
print("Entraînement avec le classificateur XGBoost : ")
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

print("\nScore du classificateur XGBoost : ")
xgb.score(X_train, y_train)

# Évaluation du Modèle



Score de l'entraînement


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
train_pred = xgb.predict(X_train)
print(accuracy_score(train_pred, y_train))

Score de test


In [ ]:
test_pred = xgb.predict(X_test)
print(accuracy_score(test_pred, y_test))

**Classification Report**

In [ ]:
print(classification_report(test_pred, y_test))

**Matrice de confusion**

In [ ]:
cm = confusion_matrix(y_test, test_pred)
print("Matrice de confusion : \n", cm)

## Notre modèle fonctionne très bien pour ce problème.
 Il n'y a pas besoin de réglage.